In [1]:
# ASF imports
import asf_search as asf

# GIS imports
import geopandas as gpd
from pyproj import Transformer

# Boken imports
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import GeoJSONDataSource, DataRange1d, Select, ColumnDataSource
from bokeh.palettes import Blues4
from bokeh.plotting import figure, show, output_file
from bokeh.models import Select
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.sampledata.sample_geojson import geojson

# Misc imports
import numpy as np
import pandas as pd
import datetime
import json

# Enable to generate Bokeh figure within Jupyter notebooks
#output_notebook()

In [2]:
# Load asf_search results into a dataframe
def make_df(results):
    df = pd.DataFrame(
    {
        'fileID':[],
        'platform':[],
        'flightDirection':[],
        'centerLat':[],
        'centerLon':[],
#         'processingLevel':[],
        'startTime':[]
    })
    name = []
    platformlist = []
    flightlist = []
    latlist = []
    lonlist = []
#     processlist = []
    stime = []
    for i in range(len(results)):
        name.append(results[i].properties['fileID'])
        platformlist.append(results[i].properties['platform'])
        flightlist.append(results[i].properties['flightDirection'])
        latlist.append(float(results[i].properties['centerLat']))
        lonlist.append(float(results[i].properties['centerLon']))
#         processlist.append(results[i].properties['processingLevel'])
        timestring = results[i].properties['startTime'][0:10] + ' ' + results[i].properties['startTime'][11:]
        stime.append(timestring)

    df.fileID = name
    df.platform = platformlist
    df.flightDirection = flightlist
    df.centerLat = latlist
    df.centerLon = lonlist
#     df.processingLevel = processlist
    df.startTime = stime
    
    return df

In [3]:
# Perform ASF search
opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'maxResults': 500,
    'processingLevel':[asf.PRODUCT_TYPE.GRD_HD, asf.PRODUCT_TYPE.GRD_MD],
    'start': '2015-01-01T00:00:00Z',
    'end': '2016-12-31T23:59:59Z'
}
results = asf.search(**opts)

df_2016 = make_df(results)
gdf = gpd.GeoDataFrame(df_2016, geometry=gpd.points_from_xy(df_2016.centerLon, df_2016.centerLat))

In [4]:
gdf.head()

,fileID,platform,flightDirection,centerLat,centerLon,startTime,geometry
0,S1B_IW_GRDH_1SDV_20161231T234437_20161231T2344...,Sentinel-1B,ASCENDING,59.2518,-91.2592,2016-12-31 23:44:37.000000,POINT (-91.25920 59.25180)
1,S1B_IW_GRDH_1SDV_20161231T234412_20161231T2344...,Sentinel-1B,ASCENDING,57.9347,-90.7849,2016-12-31 23:44:12.000000,POINT (-90.78490 57.93470)
2,S1B_IW_GRDH_1SDV_20161231T234347_20161231T2344...,Sentinel-1B,ASCENDING,56.4442,-90.2751,2016-12-31 23:43:47.000000,POINT (-90.27510 56.44420)
3,S1B_IW_GRDH_1SDV_20161231T234322_20161231T2343...,Sentinel-1B,ASCENDING,54.9525,-89.7889,2016-12-31 23:43:22.000000,POINT (-89.78890 54.95250)
4,S1B_IW_GRDH_1SDV_20161231T234257_20161231T2343...,Sentinel-1B,ASCENDING,53.4596,-89.3250,2016-12-31 23:42:57.000000,POINT (-89.32500 53.45960)


In [5]:
gdf_geometry = gdf[['fileID','geometry']]
gdf.drop(['geometry'], axis=1, inplace=True)

In [6]:
gdf.to_csv('Sentinel1_2016.csv')

In [7]:
# Convert lat/lon from WGS to Mercator to work with Bokeh plots
lats = []
for i in gdf['centerLat']:
    lats.append(i)
    
lons = []
for i in gdf['centerLon']:
    lons.append(i)
    
lonlat_to_webmercator = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

def mytransform(lon, lat):
    x, y = lonlat_to_webmercator.transform(lon, lat)
    return x, y

for i in range(len(lats)):
    lons[i], lats[i] = mytransform(lons[i],lats[i])
    
gdf['x'] = lons
gdf['y'] = lats

In [8]:
# Let's include a column to visually distinguish Sentinel-1A and B
colors = ['blue' if x=='Sentinel-1B' else 'red' for x in gdf['platform']]
gdf['colors'] = colors

In [9]:
# create a new source dataframe to be used by bokeh

gdf_source = {
    'Sentinel-1A ASCENDING':gdf[(gdf['platform']=='Sentinel-1A') & (gdf['flightDirection']=='ASCENDING')],
    'Sentinel-1A DESCENDING':gdf[(gdf['platform']=='Sentinel-1A') & (gdf['flightDirection']=='DESCENDING')],
    'Sentinel-1B ASCENDING':gdf[(gdf['platform']=='Sentinel-1B') & (gdf['flightDirection']=='ASCENDING')],
    'Sentinel-1B DESCENDING':gdf[(gdf['platform']=='Sentinel-1B') & (gdf['flightDirection']=='DESCENDING')],
    'Both ASCENDING':gdf[gdf['flightDirection']=='ASCENDING'],
    'Both DESCENDING':gdf[gdf['flightDirection']=='DESCENDING'],
    'Sentinel-1A ASCENDING+DESCENDING':gdf[gdf['platform']=='Sentinel-1A'],
    'Sentinel-1B ASCENDING+DESCENDING':gdf[gdf['platform']=='Sentinel-1B'],
    'Both ASCENDING+DESCENDING':gdf,
             }

source = ColumnDataSource(data=gdf_source['Sentinel-1A ASCENDING'])


In [10]:
tile_provider = get_provider(CARTODBPOSITRON)

p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator", plot_width=1000, plot_height = 700)


p.title.text = 'Sentinel-1 A/B overpass'
p.xaxis.axis_label = 'Longitude (degrees)'
p.yaxis.axis_label = 'Latitude (degrees)'

p.add_tile(tile_provider)
p.circle(x='x', y='y', color = 'colors',  
         size=5, alpha=0.7, source=source)

GlyphRenderer(id='1050', ...)

In [11]:
# Set up widgets
platform_select = Select(value='Sentinel-1A', title='Platform', 
                         options=['Sentinel-1A', 'Sentinel-1B', 'Both'])

flightDirection_select = Select(value='ASCENDING', title='Flight Direction', 
                                options=['ASCENDING', 'DESCENDING', 'ASCENDING+DESCENDING'])

In [12]:
# Define callback function
def update_plot(attrname, old, new):
    source.data = gdf_source[platform_select.value + ' ' + flightDirection_select.value]

        
# def update_plot_flightDir(attrname, old, new):
#     print(attrname, old, new)
#     platform = platform_select.value
#     gdf_bokeh.data = ColumnDataSource(gdf[(gdf['platform']==platform) & (gdf['flightDirection']==new)])

In [13]:
# Set up callback actions for widgets
platform_select.on_change('value', update_plot)
flightDirection_select.on_change('value', update_plot)

# Organize widgets into a column
controls = column(platform_select, flightDirection_select)

# Organize canvas into a row containing plot and widgets
curdoc().add_root(row(p, controls))
curdoc().title = "Satellites' Paths"

In [14]:
# from bokeh.models import TextInput

# def my_text_input_handler(attr, old, new):
#     print("Previous label: " + old)
#     print("Updated label: " + new)

# text_input = TextInput(value="default", title="Label:")
# text_input.on_change("value", my_text_input_handler)

In [15]:
#show(text_input)

In [16]:
# from datetime import date
# from bokeh.io import show
# from bokeh.models import CustomJS, DateRangeSlider

# def handler(new):
#     print('Radio button option ' + str(new) + ' selected.')

# date_range_slider = DateRangeSlider(value=(date(2016, 1, 1), date(2016, 12, 31)),
#                                     start=date(2016, 1, 1), end=date(2016, 12, 31))
# date_range_slider.js_on_change("value", handler)